# Inverted Pendulum

Notebook illustrating the [inverted pendulum](https://en.wikipedia.org/wiki/Inverted_pendulum) control problem (also known as cart-pole).

Based on [Control Bootcamp](https://www.youtube.com/watch?v=Pi7l8mMjYVE&list=PLMrJAkhIeNNR20Mz-VpzgfQs5zrYi085m&index=1) video lectures

In [1]:
import numpy as np
from numpy import pi, sin, cos, sqrt
import scipy
import scipy.integrate
import scipy.linalg

import matplotlib
import matplotlib.animation
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from IPython.display import HTML

sns.set_style('darkgrid')
%matplotlib inline
np.random.seed(42)

If we have a pole-cart represented by the following diagram ([_source_](https://en.wikipedia.org/wiki/Inverted_pendulum#/media/File:Cart-pendulum.svg)):

![Inverted pendulum diagram](https://i.imgur.com/bJRrpfM.png)

Then we can represent the dynamics of this system as:

$$
\begin{bmatrix}
   \dot{x} \\
   \ddot{x} \\
   \dot{\theta} \\
   \ddot{\theta}
\end{bmatrix}
=
\begin{bmatrix}
   \dot{x} \\
   \frac{-m_p l \sin(\theta) \dot{\theta}^2 + F + m_p g \cos(\theta) \sin(\theta)}{m_c + m_p - m_p \cos(\theta)^2} \\
   \dot{\theta} \\
   \frac{-m_p l \cos(\theta) \sin(\theta) \dot{\theta}^2 + F \cos(\theta) + m_p g \sin(\theta) + m_c g \sin(\theta)}{l(m_c + m_p - m_p \cos(\theta)^2 } \\
\end{bmatrix}
$$

Where:
- $x$ is the horizontal position of the cart with $\dot{x}$ the velocity and $\ddot{x}$ the acceleration:
- $\theta$ is the angle of the pole with $\theta = 0 $ the up position. $\dot{\theta}$ is the angular velocity and $\ddot{\theta}$ the angular accelration.
- $F$ is the actuation force applied to the cart in the $x$ direction.
- $l$ is the lengt of the pole.
- $m_p$ is the mass of the pole ($m$ in the figure)
- $m_c$ is the mass of the cart ($M$ in the figure)
- $g$ the gravity acting on the mass.

We can simulate this continous system by solving the [initial value problem](https://en.wikipedia.org/wiki/Initial_value_problem) of the [ordinal differntial equations](https://en.wikipedia.org/wiki/Ordinary_differential_equation) given above. We can do this with the help of the [`scipy.integrate.solve_ivp
`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) function.

In [2]:
mc = 5.  # Mass of cart (kg)
mp = 1.  # Mass of pole (kg)
l = 2.  # Length of pole (m)
g = 10.  # Gravity (m/s^2)


def dynamics(mc, mp, l, g, state, F):
    """
    Get the dynamics (derivative of state) based on the given state
        and actuating force F.

    Args:
        mc (float): Mass of cart (kg)
        mp (float): Mass of pole (kg)
        l  (float): lenght of pole (m)
        g  (float): Gravity (m/s^2)
        state (ndarray):
            x   (float): Position x (m)
            dx  (float): Horizontal velocity (m/s)
            th  (float): Angle of pole (rad)
            dth (float): Angular speed of pole (rad/s)
        F (float): Horizontal force applied to cart (kg*m/s^2)

    Returns:
        dynamics (ndarray)
        (ddx, ddth):
            ddx (float): Horizontal acceleration (m/s^2)
            ddth (float): Angular acceleration (rad/s^2)
    """
    th = state[2]
    dth = state[3]
    sin_th = sin(th)
    cos_th = cos(th)
    # intermediate variables
    mmc = mc + mp - mp * cos_th**2
    mlsinthdth2 = (mp * l * sin_th * dth**2)
    ddx = (-mlsinthdth2 + F + (mp * g * cos_th * sin_th)) / mmc 
    ddth = (
        (-mlsinthdth2 * cos_th) \
        + (F * cos_th) + (mp * g * sin_th) \
        + (mc * g * sin_th)) / (l*mmc)
    return np.asarray((state[1], ddx, dth, ddth))
        
def simulate_no_actuation(mc, mp, l, g, s0, dt, duration):
    """
    We can simulate the system  with no actuation by solving 
    the initial value problem of the ordinal differntial equations. 

    Args:
        mc (float): Mass of cart (kg)
        mp (float): Mass of pole (kg)
        l  (float): lenght of pole (m)
        g  (float): Gravity (m/s^2)
        s0 (ndarray): inital state
            x   (float): Position x (m)
            dx  (float): Horizontal velocity (m/s)
            th  (float): Angle of pole (rad)
            dth (float): Angular speed of pole (rad/s)
        dt (float): Timestep in seconds (delta)
        duration (float): Duration of simulation in seconds.
    """
    tspan = np.arange(0, duration, dt)
    # Simulate cart represented by ordinary differential equations
    return scipy.integrate.solve_ivp(
        fun=lambda t, s: dynamics(mc, mp, l, g, s, 0.),
        t_span=(0, duration), y0=s0, t_eval=tspan)

Visualise the simulation with the [matplotlib animate API](https://matplotlib.org/api/animation_api.html).

In [3]:
class SimulationPlotter:
    """Animate a simulation"""
    def __init__(self, mc, mp, l, sim):
        """
        Args:
            mc (float): Mass of cart (kg)
            mp (float): Mass of pole (kg)
            l  (float): lenght of pole (m)
            sim (object): Output of scipy.integrate.solve_ivp. 
                Simulated state over time t is stored in sim.y.
        """
        self.mc = mc
        self.mp = mp
        self.l = l
        self.x0 = 0
        self.th0 = 0
        self.sim = sim
        # Init figure
        self.fig, self.ax = plt.subplots()
        # Plotting objects
        self.cart_base = None
        self.cart = None
        self.pole = None
        self.pole_mass = None
        self.plot_artists = None
        
    def get_plot_params(self, x, th):
        """Get paramters of cart and pole visualistion based on state."""
        cart_width = 1 * sqrt(self.mc / 5)  # cart width
        cart_height = .5 * sqrt(self.mc / 5)  # cart height
        mr = .15 * sqrt(self.mp)  # pole mass radius
        # positions
        y = 0  # cart vertical position
        # Pole position
        px = x + self.l * sin(th)
        py = y + self.l * cos(th)
        return x, y, th, cart_width, cart_height, px, py, mr
        
    def init_plot(self):
        """Initialise the plot."""
        # Get current params
        x, y, th, cart_width, cart_height, px, py, mr = self.get_plot_params(
            self.x0, self.th0)
        # Plot cart
        self.cart_base, = self.ax.plot([-10, 10], [0, 0], color='black',
                linestyle='dashed', linewidth=2)
        self.cart = patches.Rectangle(
            xy=(x-cart_width / 2, y-cart_height / 2), 
            width=cart_width, height=cart_height,
            linewidth=1, edgecolor='black', facecolor='red',
            animated=True)
        self.ax.add_patch(self.cart)
        # Plot pole
        self.pole,  = self.ax.plot(
            [x, px], [y, py], 
            color='black', linewidth=2, animated=True)
        self.pole_mass = patches.Circle(
            xy=(px, py), radius=mr,
            linewidth=1, edgecolor='black', facecolor='black', animated=True)
        self.ax.add_patch(self.pole_mass)
        self.ax.axes.get_yaxis().set_visible(False)
        self.ax.set_xlim([-5, 5])
        self.ax.set_ylim([-3, 3])
        self.plot_artists = (self.cart, self.pole, self.pole_mass)
        # Avoid initial plot to stay in background
        for artist in self.plot_artists:
            artist.set_visible(False)
        return self.plot_artists
        
    def update_plot(self, x, th):
        """Update the plot with the given state"""
        # Get current params
        x, y, th, cart_width, cart_height, px, py, mr = self.get_plot_params(
            x, th)
        self.cart.set_xy((x-cart_width/2, y-cart_height/2))
        self.cart.set_width(cart_width)
        self.cart.set_height(cart_height)
        self.pole.set_data([x, px], [y, py])
        self.pole_mass.center = (px, py)
        self.pole_mass.set_radius(mr)
        return self.plot_artists
    
    def animate_step(self, i):
        """Animate a single step from the simulation."""
        if i == 0:
            for artist in self.plot_artists:
                artist.set_visible(True)
        x = self.sim.y[0, i]
        th = self.sim.y[2, i]
        self.update_plot(x, th)
        return self.plot_artists
    
    def animate(self):
        """Animate the full simulation"""
        dt = self.sim.t[1] - self.sim.t[0]
        return matplotlib.animation.FuncAnimation(
            fig=self.fig, 
            func=self.animate_step,
            init_func=self.init_plot,
            frames=self.sim.y.shape[1], 
            interval=1000*dt,
            blit=True)

In [4]:
initial_state = np.asarray([[0.], [0.], [0.5], [0.]])

# Run simulation
sim = simulate_no_actuation(
    mc, mp, l, g, 
    initial_state.flatten(), 
    dt=0.1, duration=10)

# Initialise animation plotter
plotter = SimulationPlotter(mc, mp, l, sim)
# Generate animation
ani = plotter.animate()
# Show animation
out = HTML(ani.to_jshtml())
plt.close(plotter.fig)
display(out)

## Stabilizing the pendulum

If we want to stabilize the above cart-pole system so that he pole is kept in the up ($\theta = 0$) position we need to come up with a [state-feedback](https://en.wikipedia.org/wiki/State-space_representation) controller to actuate the cart.

$$
\dot{s} = A s + B u
$$

with $s$ the state and $u=F$ the input actuation of the continous system. We also assume that what we observe the full state directly.

This means we need to linearize the non-linear system above. This can be done with the help of the first order [Taylor expansion](https://en.wikipedia.org/wiki/Taylor_series) around the point we want to stabalise.

For our system around $x=0, \dot{x}=0, \theta=0, \dot{\theta}=0$ this resuls in the following matrics of the linear dynamical system:

$$
A = \begin{bmatrix}
   0 & 1 & 0 & 0 \\
   0 & 0 & \frac{m_p g}{m_c} & 0 \\
   0 & 0 & 0 & 1 \\
   0 & 0 & \frac{(m_p + m_c) g}{m_c l} & 0
\end{bmatrix}
\;
B = \begin{bmatrix}
   0 \\
   \frac{1}{m_c} \\
   0 \\
   \frac{1}{m_c l}
\end{bmatrix}
$$

For a continous system we can check if this system is unstable if any of the eigenvalues of $A$ is larger than 0. 

In [5]:
A = np.asarray([
    [0., 1., 0.,                 0.],
    [0., 0., (mp*g)/mc,          0.],
    [0., 0., 0.,                 1.],
    [0., 0., ((mp+mc)*g)/(mc*l), 0.]
])

B = np.asarray([
    [0.],
    [1./mc],
    [0.],
    [1/(mc*l)]
])

if (np.linalg.eig(A)[0] > 0).any():
    print('System is unstable (at least 1 eigenvalue > 0)')
else:
    print('System is stable (all eigenvalues <= 0)')

System is unstable (at least 1 eigenvalue > 0)


This system can be stabilized around $\theta = 0$ by adding control in the form of $u = -K s$, wich means that:

$$
\dot{s} = A s - B K s = (A - B K) s
$$

With $K$ the control matrix. Setting $K$ allows to change the values that update the state $s$. We can change these values arbitrarily with the help of the [Python Control Systems Library](https://python-control.readthedocs.io/en/0.8.0/index.html)'s [`place`](https://python-control.readthedocs.io/en/0.8.0/generated/control.place.html) function. However this might result in a to agressive system so we are going to set K with the help of a regulator.

## Linear-quadratic regulator

The [linear-quadratic regulator](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator) (LQR) allows to set a [state-feedback](https://en.wikipedia.org/wiki/State-space_representation) controller to control a linear dynamical system.

LQR optimizes a quadratic cost function based on a weight matrix $Q$ and input weight $R$. $Q$ defines the cost for the deviance of each state paramter from the target. $R$ defines the cost of the input control for the actuating force $F$.

In [6]:
Q = np.asarray([
    [1., 0., 0.,  0.],
    [0., 1., 0.,  0.],
    [0., 0., 10., 0.],
    [0., 0., 0.,  100.]
])
R = np.asarray([[.1]])

In [7]:
def lqr(A, B, Q, R):
    """LQR solver. Returns the gain matrix to control the system."""
    X = scipy.linalg.solve_continuous_are(A, B, Q, R)
    K = np.linalg.solve(R, (B.T @ X))
    return K


# Solve the control gain matrix with LQR
K = lqr(A, B, Q, R)

if (np.linalg.eig((A - (B @ K)))[0] > 0).any():
    print('System is unstable (at least 1 eigenvalue > 0)')
else:
    print('System is stable (all eigenvalues <= 0)')

System is stable (all eigenvalues <= 0)


In [8]:
def simulate_lqr(mc, mp, l, g, s0, K, dt, duration):
    """
    Simulate the system controlled by LQR.

    Args:
        mc (float): Mass of cart (kg)
        mp (float): Mass of pole (kg)
        l  (float): lenght of pole (m)
        g  (float): Gravity (m/s^2)
        s0 (ndarray): inital state
            x   (float): Position x (m)
            dx  (float): Horizontal velocity (m/s)
            th  (float): Angle of pole (rad)
            dth (float): Angular speed of pole (rad/s)
        K (ndarray): Control gain matrix from LQR solver.
        dt (float): Timestep in seconds (delta)
        duration (float): Duration of simulation in seconds.
    """
    tspan = np.arange(0, duration, dt)
    # Simulate cart represented by ordinary differential equations
    return scipy.integrate.solve_ivp(
        fun=lambda t, s: dynamics(mc, mp, l, g, s, -K@s),
        t_span=(0, duration), y0=s0, t_eval=tspan)


# Run simulation
sim_lqr = simulate_lqr(
    mc, mp, l, g, 
    initial_state.flatten(), K,
    dt=0.1, duration=12)

# Initialise animation plotter
plotter_lqr = SimulationPlotter(mc, mp, l, sim_lqr)
# Generate animation
ani = plotter_lqr.animate()
# Show animation
out = HTML(ani.to_jshtml())
plt.close(plotter_lqr.fig)
display(out)